In [ ]:

# Convert all feature columns to numeric, coerce errors to NaN
X_train = pd.DataFrame(X_train, columns=['q', 'month', 'day_of_year', 'is_weekend'])
X_test = pd.DataFrame(X_test, columns=['q', 'month', 'day_of_year', 'is_weekend'])

X_train = X_train.apply(pd.to_numeric, errors='coerce')
X_test = X_test.apply(pd.to_numeric, errors='coerce')

# Drop rows with NaNs
X_train.dropna(inplace=True)
X_test.dropna(inplace=True)

# Align target and original data
y_train = y_train[X_train.index]
y_test = y_test[X_test.index]
train_data = train_data.iloc[X_train.index]
test_data = test_data.iloc[X_test.index]


# Scale data
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()
X_train = scaler_X.fit_transform(X_train)
y_train = scaler_y.fit_transform(y_train)
X_test = scaler_X.transform(X_test)
y_test = scaler_y.transform(y_test)

# Define different configurations for neurons per layer
configurations = [
    ([2, 1], '2, 1'),             
    ([8, 4], '8, 4'),             
    ([32, 16, 8], '32, 16, 8'),   
    ([64, 32, 16], '64, 32, 16'), 
    ([64, 32, 16, 8], '64, 32, 16, 8')  
]

# Define different configurations for activation functions
activations = [
    ('sigmoid', 'Sigmoid'),
    ('tanh', 'Tanh'),
    ('relu', 'ReLU'),
    ('elu', 'ELU'),
    ('selu', 'SELU')  
]

# Define different configurations for optimizers
optimizers = [
    (SGD(learning_rate=0.01), 'SGD'),
    (RMSprop(learning_rate=0.001), 'RMSprop'),
    (Adam(learning_rate=0.001), 'Adam'),
    (Adagrad(learning_rate=0.01), 'Adagrad'),
    (Adadelta(learning_rate=1.0), 'Adadelta')  
]

# Function to build and compile the neural network model
def build_nn_model(neurons_per_layer, activation_function, optimizer):
    model = Sequential()
    model.add(Dense(units=neurons_per_layer[0], activation=activation_function, input_dim=X_train.shape[1]))
    for neurons in neurons_per_layer[1:]:
        model.add(Dense(units=neurons, activation=activation_function))
    model.add(Dense(units=1, activation='linear'))

    model.compile(optimizer=optimizer, loss='mean_squared_error')
    return model

results = []

# Implement logging and monitoring
model_checkpoint_callback = ModelCheckpoint(
    filepath='model_checkpoint.h5', 
    save_weights_only=False,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

csv_logger = CSVLogger('training_log.csv', append=True)